In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


This makes the plot demonstrating the relationship between the mean and eccentric anomaly

In [2]:
eccs = 0., 0.2, 0.5, 0.9, 0.99
t = np.linspace(0, 2*np.pi)
for e in eccs:
    plt.plot(t, t - e*np.sin(t), label=r"$e=%g$"%e)
plt.ylabel(r"Mean Anomaly $=2 \pi t / T$")
plt.xlabel(r"Eccentric Anomaly $\xi$")
plt.legend()
plt.plot(t, np.repeat(np.random.rand()*2*np.pi,len(t)),color='black',ls='dashed')
plt.xlim(0,2*np.pi)
plt.ylim(0,2*np.pi)
plt.savefig("Anomaly.pdf",bbox_inches='tight')

Defining functions for the three different rootfind methods, which return both the final answer and the sequence of error estimators.

In [3]:
def SecantMethod(func, x0, x1, errtol=1e-15):
    err = 1e100
    errs = []
    while err > errtol:
        y0, y1 = func(x0), func(x1)
        x2 = x1 - y1*(x1-x0)/(y1 - y0)
        x0, x1 = x1, x2
        err = abs((x0-x1)/x1) 
        errs.append(err)
    return x1, errs

def BisectionMethod(func, x0, x1, errtol=1e-15):
    # First check whether the initial guesses bracket the root
    y0, y1 = func(x0), func(x1)
    if not y0*y1 < 0: 
        raise ValueError("Initial guesses do not bracket the root!")
        return
    errs = []
    err = 1e100
    #Now we iterate to pare the error down
    while err > errtol:
        x2 = (x0+x1)*0.5  # midpoint
        y2 = func((x0+x1)*0.5)  # function value at the midpoint
        if y0*y2 < 0:
            x0, x1 = x0, x2
            y0, y1 = y0, y2
        else:
            x0, x1 = x1, x2
        err = abs((x0-x1)/x1)
        errs.append(err)
    return x1, errs

def NewtonMethod(func, dfunc, x0, errtol=1e-15):
    err = 1e100
    errs = []
    while err > errtol:
        x1 = x0 - func(x0)/dfunc(x0)
        err = abs((x1-x0)/x1)
        x0 = x1
        errs.append(err)
    return x1, errs
        

Let's make a plot of the convergence rate by plotting ε<sub>i+1</sub> as a function of ε<sub>i</sub>

In [5]:
func = lambda x: np.sin(x) - 0.5
dfunc = lambda x: np.cos(x)
sol1, errs1 = SecantMethod(func, 0.4,0.7)
sol2, errs2 = BisectionMethod(func, 0.4,0.7)
sol3, errs3 = NewtonMethod(func, dfunc, 0.6)
plt.loglog(errs1[:-1],errs1[1:], label = "Secant")
plt.loglog(errs2[:-1],errs2[1:], label = "Bisection")
plt.loglog(errs3[:-1],errs3[1:], label = "Newton")
plt.xlabel("$\epsilon_{i}$")
plt.ylabel("$\epsilon_{i+1}$")
plt.legend()
plt.savefig("Convergence.pdf", bbox_inches='tight')

Now let's create a function that computes the eccentric anomaly as a function of the mean anomaly, using Newton iterations. Here I use Numba's @vectorize decorator to turn the function into a jit-compiled numpy ufunc, which runs reasonably quickly. Then I define a function that gives the polar coordinates of the orbit as a function of time, returning t, xi, and r.

In [8]:
from numba import float32, float64, vectorize

@vectorize([float32(float32, float32),
            float64(float64, float64)])
def EccentricAnomaly(mean_anomaly, ecc):
    x0 = mean_anomaly
    err = 1e100
    N = 0
    TWOPI = 2*np.pi
    while abs(err/TWOPI) > 2e-15: # do Newton iterations
        err = (x0 - ecc*np.sin(x0) - mean_anomaly)/(1 - ecc*np.cos(x0)) 
        x0 -= err
        x0 = x0%(TWOPI)
        N+=1
        if N>1000: print(x0, err/TWOPI)
    return x0

def Orbit(a, T, e, N = 10000):
    mean_anomaly = np.arange(0, 4*np.pi, 4*np.pi/N) % (2*np.pi)
    xi = EccentricAnomaly(mean_anomaly, e)
    t = T * np.arange(0,1,1./N)
    r = a * (1 - e * np.cos(xi))
    x = a * (np.cos(xi) - e)
    y = a * (1 - e*e)**0.5 * np.sin(xi)
    return t, xi, r, x, y

Hulse-Taylor binary parameters in s - km units

In [6]:
e = 0.617139
T = 27906.98161
a = 2.34186 * 3e5

Just do a plot of the orbit, uncorrected for angle

In [11]:
t, xi, r, x, y = Orbit(a, T, e)
plt.plot(x, y)
plt.axes().set_aspect('equal')
plt.xlabel("X (km)")
plt.ylabel("Y (km)")
plt.savefig("Orbit.pdf", bbox_inches='tight')

Now let's do a bunch of plots for different angles and see which one looks the most reasonable

In [24]:
vx, vy = np.gradient(x, t), np.gradient(y, t)

sols = np.arange(0,2*np.pi, 2*np.pi/8)
for s in sols:
    plt.plot(t/T, np.cos(s) * vx + np.sin(s)*vy, label="$\phi=%g$"%s)
plt.legend(frameon=True)
plt.xlabel("Phase")
plt.ylabel("Radial Velocity ($\rm{km\,s^{-1}}$)")
plt.savefig("RadialVelocity.pdf", bbox_inches='tight')

Looks like the best fit is ~4.7 rad!